In [1]:
# Import dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# Store the CSV into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Iqaluit,CA,12/24/2020 2:54,63.75,-68.51,-0.40,70,75,18.34
1,1,Atuona,PF,12/24/2020 2:58,-9.80,-139.03,78.64,76,4,16.71
2,2,Punta Arenas,CL,12/24/2020 2:58,-53.15,-70.92,44.01,70,2,25.28
3,3,Guerrero Negro,MX,12/24/2020 2:58,27.98,-114.06,62.08,65,100,5.95
4,4,Fairlie,GB,12/24/2020 2:58,55.76,-4.86,35.01,79,46,9.17


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get only positive max temperatures.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                dissipating=False, max_intensity=300, point_radius=6)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity.
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=6)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness.
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=6)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent wind speed.
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=6)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter city data to find cities that fit temperature criteria.
vacation_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                               (city_data_df["Max Temp"] >= min_temp)]
vacation_df.dropna
vacation_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Atuona,PF,12/24/2020 2:58,-9.80,-139.03,78.64,76,4,16.71
12,12,Butaritari,KI,12/24/2020 2:58,3.07,172.79,81.23,77,68,15.68
17,17,Kavieng,PG,12/24/2020 2:58,-2.57,150.80,85.95,66,91,5.68
20,20,Carnarvon,AU,12/24/2020 2:58,-24.87,113.63,80.60,83,0,11.41
25,25,Kilindoni,TZ,12/24/2020 2:58,-7.91,39.67,81.57,79,36,11.07
29,29,Neiafu,TO,12/24/2020 2:58,-18.65,-173.98,84.20,74,40,5.82
36,36,Miri,MY,12/24/2020 2:54,4.41,114.01,86.00,79,40,6.93
37,37,Acajutla,SV,12/24/2020 2:58,13.59,-89.83,78.44,73,0,5.19
40,40,Muisne,EC,12/24/2020 2:58,0.60,-80.03,75.45,87,100,7.58
46,46,Hilo,US,12/24/2020 2:58,19.73,-155.09,80.60,51,1,8.05


In [12]:
# Create a DataFrame to store hotel names along with some city information.
hotel_df = vacation_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,78.64,-9.80,-139.03,
12,Butaritari,KI,81.23,3.07,172.79,
17,Kavieng,PG,85.95,-2.57,150.80,
20,Carnarvon,AU,80.60,-24.87,113.63,
25,Kilindoni,TZ,81.57,-7.91,39.67,


In [20]:
# Set the parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [21]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the param dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make a request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [28]:
# Store hotel_info
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=6)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
                                  
# Call the figure.
fig

Figure(layout=FigureLayout(height='420px'))